NLP Processing

In [129]:
!pip install --upgrade pip
!pip install re
!pip install typer
!pip install spacy
!pip install scispacy

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try apt install
    python3-xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Debian-packaged Python package,
    create a virtual environment using python3 -m venv path/to/venv.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip. Make
    sure you have python3-full installed.
    
    If you wish to install a non-Debian packaged Python application,
    it may be easiest to use pipx install xyz, which will manage a
    virtual environment for you. Make sure you have pipx installed.
    
    See /usr/share/doc/python3.12/README.venv for more information.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detai

In [133]:
import pandas as pd
import spacy

In [91]:
import pandas as pd

# Load your file
disease_chems = pd.read_csv('/home/giannis-charalampopoulos/M2PHDS/web based data/wbd-project/CTD_disease_chems_curated.csv')

# Split the PubMedIDs column into lists
disease_chems['PubMedIDs'] = disease_chems['PubMedIDs'].astype(str).str.split('|')

# Explode into multiple rows
disease_chems = disease_chems.explode('PubMedIDs')

# Reset index (optional)
disease_chems = disease_chems.reset_index(drop=True)

# View result
disease_chems.head()

,# Input,ChemicalName,ChemicalID,CasRN,DiseaseName,DiseaseID,DiseaseCategories,OmimIDs,PubMedIDs,DirectEvidence
0,mesh:d002659,succinyladenosine,C043565,4542-23-8,Adenylosuccinate lyase deficiency,MESH:C538235,Genetic disease (inborn)|Mental disorder|Metab...,NaN,10888601,marker/mechanism
1,mesh:d002659,succinyladenosine,C043565,4542-23-8,Adenylosuccinate lyase deficiency,MESH:C538235,Genetic disease (inborn)|Mental disorder|Metab...,NaN,10958654,marker/mechanism
2,mesh:d002659,succinyladenosine,C043565,4542-23-8,Adenylosuccinate lyase deficiency,MESH:C538235,Genetic disease (inborn)|Mental disorder|Metab...,NaN,17485188,marker/mechanism
3,mesh:d002659,succinyladenosine,C043565,4542-23-8,Adenylosuccinate lyase deficiency,MESH:C538235,Genetic disease (inborn)|Mental disorder|Metab...,NaN,18368630,marker/mechanism
4,mesh:d002659,succinyladenosine,C043565,4542-23-8,Adenylosuccinate lyase deficiency,MESH:C538235,Genetic disease (inborn)|Mental disorder|Metab...,NaN,23055421,marker/mechanism


In [92]:
disease_chems

,# Input,ChemicalName,ChemicalID,CasRN,DiseaseName,DiseaseID,DiseaseCategories,OmimIDs,PubMedIDs,DirectEvidence
0,mesh:d002659,succinyladenosine,C043565,4542-23-8,Adenylosuccinate lyase deficiency,MESH:C538235,Genetic disease (inborn)|Mental disorder|Metab...,NaN,10888601,marker/mechanism
1,mesh:d002659,succinyladenosine,C043565,4542-23-8,Adenylosuccinate lyase deficiency,MESH:C538235,Genetic disease (inborn)|Mental disorder|Metab...,NaN,10958654,marker/mechanism
2,mesh:d002659,succinyladenosine,C043565,4542-23-8,Adenylosuccinate lyase deficiency,MESH:C538235,Genetic disease (inborn)|Mental disorder|Metab...,NaN,17485188,marker/mechanism
3,mesh:d002659,succinyladenosine,C043565,4542-23-8,Adenylosuccinate lyase deficiency,MESH:C538235,Genetic disease (inborn)|Mental disorder|Metab...,NaN,18368630,marker/mechanism
4,mesh:d002659,succinyladenosine,C043565,4542-23-8,Adenylosuccinate lyase deficiency,MESH:C538235,Genetic disease (inborn)|Mental disorder|Metab...,NaN,23055421,marker/mechanism
...,...,...,...,...,...,...,...,...,...,...
875,mesh:d001321,Vitamin B 12,D014805,68-19-9,Autistic Disorder,MESH:D001321,Mental disorder,NaN,24676906,marker/mechanism
876,mesh:d001321,Vitamin B 6,D025101,8059-24-3,Autistic Disorder,MESH:D001321,Mental disorder,NaN,16494569,marker/mechanism|therapeutic
877,mesh:d001321,Vitamin B 6,D025101,8059-24-3,Autistic Disorder,MESH:D001321,Mental disorder,NaN,16846101,marker/mechanism|therapeutic
878,mesh:d001321,Zinc,D015032,7440-66-6,Autistic Disorder,MESH:D001321,Mental disorder,NaN,28569757,marker/mechanism


In [93]:
pmid_list=disease_chems['PubMedIDs'].astype(str).tolist()

In [96]:
len(pmid_list)

208

In [97]:
import requests
import time
import pandas as pd
from xml.etree import ElementTree as ET

# Your list of PMIDs
pmid_list = list(set(pmid_list))  # remove duplicates
total = len(pmid_list)

def fetch_pubmed_xml(pmid):
    """Fetch a single PubMed record via NCBI EFetch."""
    url = (
        "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
        f"?db=pubmed&id={pmid}&retmode=xml"
    )

    response = requests.get(url)
    if response.status_code != 200:
        print(f"⚠️ Failed to fetch {pmid}")
        return None

    return response.text

records = []
for idx, pmid in enumerate(pmid_list, start=1):

    xml_data = fetch_pubmed_xml(pmid)
    if xml_data is None:
        print(f"❌ Could not retrieve PMID {pmid} ({idx}/{total})")
        continue

    # Parse XML
    root = ET.fromstring(xml_data)
    title = root.findtext(".//ArticleTitle")
    abstract = " ".join(
        [t.text for t in root.findall(".//AbstractText") if t.text]
    )

    records.append({
        "pmid": pmid,
        "title": title,
        "abstract": abstract,
        "xml_raw": xml_data
    })

    # Print progress for every PMID found
    print(f"✔️  Retrieved {idx}/{total} → PMID {pmid}")

    # Optional milestone print every 25 items
    if idx % 25 == 0:
        print(f"⏳ Progress: {idx}/{total} ({100*idx/total:.1f}%)")

    # polite delay (NCBI recommends <10 req/sec)
    time.sleep(0.15)

print("DONE! All available PMIDs retrieved.")
df = pd.DataFrame(records)
print("Total records downloaded:", len(df))

df.head()


✔️  Retrieved 1/208 → PMID 22537663
✔️  Retrieved 2/208 → PMID 25286049
✔️  Retrieved 3/208 → PMID 18278980
✔️  Retrieved 4/208 → PMID 18387724
✔️  Retrieved 5/208 → PMID 17150311
✔️  Retrieved 6/208 → PMID 18080171
✔️  Retrieved 7/208 → PMID 18929414
✔️  Retrieved 8/208 → PMID 6502317
✔️  Retrieved 9/208 → PMID 12368987
✔️  Retrieved 10/208 → PMID 29120534
✔️  Retrieved 11/208 → PMID 12607287
✔️  Retrieved 12/208 → PMID 9808916
✔️  Retrieved 13/208 → PMID 18062943
✔️  Retrieved 14/208 → PMID 8157157
✔️  Retrieved 15/208 → PMID 8808733
✔️  Retrieved 16/208 → PMID 18026891
✔️  Retrieved 17/208 → PMID 8570775
✔️  Retrieved 18/208 → PMID 18558336
✔️  Retrieved 19/208 → PMID 37207798
✔️  Retrieved 20/208 → PMID 19393386
✔️  Retrieved 21/208 → PMID 39194383
✔️  Retrieved 22/208 → PMID 23613074
✔️  Retrieved 23/208 → PMID 15820420
✔️  Retrieved 24/208 → PMID 17690064
✔️  Retrieved 25/208 → PMID 15930063
⏳ Progress: 25/208 (12.0%)
✔️  Retrieved 26/208 → PMID 19755133
✔️  Retrieved 27/208 → PM

,pmid,title,abstract,xml_raw
0,22537663,Di-(2-ethylhexyl) phthalate and autism spectru...,ASDs (autism spectrum disorders) are a complex...,"<?xml version=""1.0"" ?>\n<!DOCTYPE PubmedArticl..."
1,25286049,"Particulate matter exposure, prenatal and post...",Recent studies suggest that exposure to traffi...,"<?xml version=""1.0"" ?>\n<!DOCTYPE PubmedArticl..."
2,18278980,Spotlight on risperidone in irritability assoc...,"Risperidone (Risperdal), a psychotropic atypic...","<?xml version=""1.0"" ?>\n<!DOCTYPE PubmedArticl..."
3,18387724,Risperidone-induced double incontinence.,,"<?xml version=""1.0"" ?>\n<!DOCTYPE PubmedArticl..."
4,17150311,Spironolactone might be a desirable immunologi...,Multiple studies now demonstrate that autism i...,"<?xml version=""1.0"" ?>\n<!DOCTYPE PubmedArticl..."


In [98]:
pubmed_dataset=df

In [99]:
pubmed_dataset=pubmed_dataset[['pmid', 'abstract']]
pubmed_dataset



,pmid,abstract
0,22537663,ASDs (autism spectrum disorders) are a complex...
1,25286049,Recent studies suggest that exposure to traffi...
2,18278980,"Risperidone (Risperdal), a psychotropic atypic..."
3,18387724,
4,17150311,Multiple studies now demonstrate that autism i...
...,...,...
203,20013426,Functional changes in the brains of autistic c...
204,15246500,
205,32661462,Polychlorinated biphenyls (PCBs) and organochl...
206,21156395,Little is known about environmental causes and...


In [100]:
pubmed_dataset = pubmed_dataset[
    pubmed_dataset["abstract"].astype(str).str.strip() != ""
]


In [101]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag

# Downloads
nltk.download("punkt")
nltk.download("averaged_perceptron_tagger")
nltk.download("wordnet")

lemmatizer = WordNetLemmatizer()

# Function to convert nltk pos tags to wordnet tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

# Tokenize + POS tag + Lemmatize
pubmed_dataset["tokens"] = (
    pubmed_dataset["abstract"]
        .astype(str)
        .str.lower()
        .apply(word_tokenize)
        .apply(lambda words: pos_tag(words))
        .apply(lambda tagged: [
            lemmatizer.lemmatize(w, get_wordnet_pos(tag))
            for w, tag in tagged
        ])
)


[nltk_data] Downloading package punkt to /home/giannis-
[nltk_data]     charalampopoulos/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/giannis-charalampopoulos/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/giannis-
[nltk_data]     charalampopoulos/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/tmp/ipykernel_24046/988891669.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pubmed_dataset["tokens"] = (


In [102]:
import re

def custom_tokenize(text):
    # keep numbers like 15,000 or 3.5e10 as one token
    pattern = r'\d[\d,\.]*|\w+'
    return re.findall(pattern, text.lower())


In [103]:

pubmed_dataset["tokens"]

0      [asds, (, autism, spectrum, disorder, ), be, a...
1      [recent, study, suggest, that, exposure, to, t...
2      [risperidone, (, risperdal, ), ,, a, psychotro...
4      [multiple, study, now, demonstrate, that, auti...
5      [the, aim, of, the, study, be, to, compare, sa...
                             ...                        
202    [the, association, between, prenatal, phthalat...
203    [functional, change, in, the, brain, of, autis...
205    [polychlorinated, biphenyls, (, pcbs, ), and, ...
206    [little, be, know, about, environmental, cause...
207    [an, open-label, retrospective, study, be, con...
Name: tokens, Length: 203, dtype: object

In [104]:
pubmed_dataset["tokens"] = pubmed_dataset["tokens"].apply(
    lambda lst: [t for t in lst if any(c.isalnum() for c in t)]
)

/tmp/ipykernel_24046/3246752615.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pubmed_dataset["tokens"] = pubmed_dataset["tokens"].apply(


In [105]:
pubmed_dataset=pubmed_dataset.rename(columns={"tokens":"Text"})

In [106]:
pubmed_dataset_processed=pubmed_dataset[["pmid", "Text"]]


In [107]:
merged = pd.merge(
    pubmed_dataset_processed,
    disease_chems,
    left_on="pmid",
    right_on="PubMedIDs",
    how="inner"
)


In [110]:
merged

,pmid,Text,# Input,ChemicalName,ChemicalID,CasRN,DiseaseName,DiseaseID,DiseaseCategories,OmimIDs,PubMedIDs,DirectEvidence
0,22537663,"[asds, autism, spectrum, disorder, be, a, comp...",mesh:d002659,mono(2-ethyl-5-oxohexyl)phthalate,C080276,40321-98-0,Autism Spectrum Disorder,MESH:D000067877,Mental disorder,NaN,22537663,marker/mechanism
1,22537663,"[asds, autism, spectrum, disorder, be, a, comp...",mesh:d000067877,mono(2-ethyl-5-oxohexyl)phthalate,C080276,40321-98-0,Autism Spectrum Disorder,MESH:D000067877,Mental disorder,NaN,22537663,marker/mechanism
2,25286049,"[recent, study, suggest, that, exposure, to, t...",mesh:d002659,Air Pollutants,D000393,NaN,Autistic Disorder,MESH:D001321,Mental disorder,NaN,25286049,marker/mechanism
3,25286049,"[recent, study, suggest, that, exposure, to, t...",mesh:d002659,Particulate Matter,D052638,NaN,Autistic Disorder,MESH:D001321,Mental disorder,NaN,25286049,marker/mechanism
4,25286049,"[recent, study, suggest, that, exposure, to, t...",mesh:d002659,Vehicle Emissions,D001335,NaN,Autistic Disorder,MESH:D001321,Mental disorder,NaN,25286049,marker/mechanism
...,...,...,...,...,...,...,...,...,...,...,...,...
860,32661462,"[polychlorinated, biphenyls, pcbs, and, organo...",mesh:d000067877,PCB 180,C410127,NaN,Autism Spectrum Disorder,MESH:D000067877,Mental disorder,NaN,32661462,marker/mechanism
861,21156395,"[little, be, know, about, environmental, cause...",mesh:d002659,Air Pollutants,D000393,NaN,Autistic Disorder,MESH:D001321,Mental disorder,NaN,21156395,marker/mechanism
862,21156395,"[little, be, know, about, environmental, cause...",mesh:d000067877,Air Pollutants,D000393,NaN,Autistic Disorder,MESH:D001321,Mental disorder,NaN,21156395,marker/mechanism
863,21156395,"[little, be, know, about, environmental, cause...",mesh:d001321,Air Pollutants,D000393,NaN,Autistic Disorder,MESH:D001321,Mental disorder,NaN,21156395,marker/mechanism


In [191]:
import pandas as pd
from collections import Counter
import math

final_dataset = merged  # your dataframe

# ===================== USE YOUR EXACT PATTERNS =====================
RELATION_PATTERNS = [
    ["causes"],
    ["induces"],
    ["associated", "with"],
    ["related", "to"],
    ["leads", "to"],
    ["risk", "of"],
    ["increases", "risk"],
    ["linked", "to"]
]

# ========= ASSIGN WEIGHTS BASED ON TYPE + LENGTH =========

def get_weight(pattern):
    """Weighted pattern scoring: multi-word bonus + type importance."""
    base = 1.0 + (0.6 * (len(pattern) - 1))  # multi-word bonus
    text = " ".join(pattern)

    # Strong causal relationships
    if any(word in text for word in ["causes", "induces", "leads"]):
        return base + 2.0

    # Increased measured risk
    if "risk" in text or "increases" in text:
        return base + 1.5

    # General correlation (weaker)
    if any(word in text for word in ["associated", "linked", "related"]):
        return base + 0.7

    return base

PATTERN_WEIGHTS = {tuple(p): get_weight(p) for p in RELATION_PATTERNS}


def match_pattern_weight(tokens):
    total_weight = 0
    for pattern, weight in PATTERN_WEIGHTS.items():
        n = len(pattern)
        for i in range(len(tokens) - n + 1):
            if tuple(tokens[i:i+n]) == pattern:
                total_weight += weight
    return total_weight


# ===================== BUILD COUNTS =====================

pair_counts = Counter()
pair_pattern_score = Counter()

for i, row in final_dataset.iterrows():
    chem = row["ChemicalName"]
    dis  = row["DiseaseName"]
    tokens = row["Text"]  # token list
    pair = (chem, dis)

    pair_counts[pair] += 1
    pair_pattern_score[pair] += match_pattern_weight(tokens)


# ===================== FINAL RESULT =====================

rows = []
for (chem, dis), count in pair_counts.items():
    pattern_score = pair_pattern_score[(chem, dis)]

    # Stronger EvidenceScore: combines impact + frequency
    evidence_score = math.log(pattern_score + 1) * math.log(count + 2)

    rows.append({
        "ChemicalName": chem,
        "DiseaseName": dis,
        "CoOccurrences": count,
        "WeightedPatternEvidence": pattern_score,
        "EvidenceScore": evidence_score
    })

result = pd.DataFrame(rows).sort_values("EvidenceScore", ascending=False)
print(result)



                    ChemicalName                             DiseaseName  \
14                 Valproic Acid                       Autistic Disorder   
17                 Valproic Acid                Autism Spectrum Disorder   
4                    Risperidone                       Autistic Disorder   
67        Diethylhexyl Phthalate                Autism Spectrum Disorder   
6                    Haloperidol                       Autistic Disorder   
..                           ...                                     ...   
75       5-hydroxymethylcytosine                       Autistic Disorder   
76              5-Methylcytosine                       Autistic Disorder   
77   8-Hydroxy-2'-Deoxyguanosine                       Autistic Disorder   
78           25-hydroxyvitamin D                Autism Spectrum Disorder   
181                   Topiramate  Child Development Disorders, Pervasive   

     CoOccurrences  WeightedPatternEvidence  EvidenceScore  
14              81        

In [192]:
result

,ChemicalName,DiseaseName,CoOccurrences,WeightedPatternEvidence,EvidenceScore
14,Valproic Acid,Autistic Disorder,81,74.4,19.101796
17,Valproic Acid,Autism Spectrum Disorder,26,49.6,13.075409
4,Risperidone,Autistic Disorder,78,18.6,13.038850
67,Diethylhexyl Phthalate,Autism Spectrum Disorder,10,24.8,8.076877
6,Haloperidol,Autistic Disorder,12,18.6,7.852593
...,...,...,...,...,...
75,5-hydroxymethylcytosine,Autistic Disorder,3,0.0,0.000000
76,5-Methylcytosine,Autistic Disorder,3,0.0,0.000000
77,8-Hydroxy-2'-Deoxyguanosine,Autistic Disorder,3,0.0,0.000000
78,25-hydroxyvitamin D,Autism Spectrum Disorder,4,0.0,0.000000


NLP-Based Extraction of Neurodevelopmental Associations

In [155]:
import pandas as pd
import spacy
import math
from collections import defaultdict, Counter

# ================== NLP SETUP ===================
nlp = spacy.load("en_core_web_sm")

# VERB CATEGORIES
CAUSE = {"cause", "induce", "trigger", "increase", "promote"}
TREAT = {"reduce", "decrease", "treat", "prevent", "protect", "suppress"}
ASSOC = {"associate", "link", "correlate", "relate", "connect"}

def classify_relation(verb):
    v = verb.lemma_.lower()
    if v in CAUSE:  return "CAUSE"
    if v in TREAT:  return "TREAT/PROTECT"
    if v in ASSOC:  return "ASSOCIATION"
    return None

# ----- Relation Extraction -----
def extract_relation(chem, dis, sentence):
    if not isinstance(sentence, str):
        sentence = " ".join(sentence)

    sentence = sentence.lower()
    chem = chem.lower()
    dis  = dis.lower()

    if chem not in sentence or dis not in sentence:
        return None

    doc = nlp(sentence)

    for token in doc:
        rel = classify_relation(token)
        if rel:
            if any(child.dep_ == "neg" for child in token.children):
                return None
            return rel
    return None

# ================== COUNT RELATIONS ===================
relations = defaultdict(int)

for i, row in merged.iterrows():
    rel = extract_relation(row["ChemicalName"], row["DiseaseName"], row["Text"])
    if rel:
        relations[(row["ChemicalName"], row["DiseaseName"], rel)] += 1

result = pd.DataFrame([
    {"ChemicalName": a, "DiseaseName": b, "Relation": r, "Occurrences": c}
    for (a, b, r), c in relations.items()
])

# ================== PMI SCORE ===================
pairs = Counter()
chem_counts = Counter()
dis_counts = Counter()

total = len(merged)

for i, row in merged.iterrows():
    chem = row["ChemicalName"]
    dis  = row["DiseaseName"]
    pairs[(chem, dis)] += 1
    chem_counts[chem] += 1
    dis_counts[dis] += 1

pmi_values = {}

for (chem, dis), freq in pairs.items():
    p_chem = chem_counts[chem] / total
    p_dis  = dis_counts[dis] / total
    p_pair = freq / total
    pmi = math.log((p_pair) / (p_chem * p_dis) + 1e-10)
    pmi_values[(chem, dis)] = pmi

result["PMI_Score"] = result.apply(
    lambda row: pmi_values.get((row["ChemicalName"], row["DiseaseName"]), 0),
    axis=1
)









In [156]:
result

,ChemicalName,DiseaseName,Relation,Occurrences,PMI_Score
0,Risperidone,Autistic Disorder,ASSOCIATION,12,0.305839
1,Haloperidol,Autistic Disorder,CAUSE,6,0.355850
2,Risperidone,Autistic Disorder,CAUSE,6,0.305839
3,Valproic Acid,Autism Spectrum Disorder,CAUSE,14,0.011358
4,Naltrexone,Autistic Disorder,CAUSE,3,0.355850
5,Risperidone,Autistic Disorder,TREAT/PROTECT,15,0.305839
6,Chlorpyrifos,Autism Spectrum Disorder,ASSOCIATION,4,1.444610
7,Acetaminophen,Autism Spectrum Disorder,CAUSE,2,0.528319
8,Cyproheptadine,Autistic Disorder,CAUSE,3,0.355850
9,"1,3-butadiene",Autistic Disorder,ASSOCIATION,3,0.355850
